# Acting Out

In [309]:
data = pd.read_csv('./data/tmdb_5000_credits.csv')

In [333]:
import ast

cast_df = pd.concat([
    pd.concat([pd.DataFrame(ast.literal_eval(x.cast)), pd.DataFrame({'movie' : [x.title]*len(ast.literal_eval(x.cast))})], axis = 1) for ndx, x in data.iterrows()
], axis = 0)

In [334]:
cast_df

,cast_id,character,credit_id,gender,id,name,order,movie
0,242.0,Jake Sully,5602a8a7c3a3685532001c9a,2.0,65731.0,Sam Worthington,0.0,Avatar
1,3.0,Neytiri,52fe48009251416c750ac9cb,1.0,8691.0,Zoe Saldana,1.0,Avatar
2,25.0,Dr. Grace Augustine,52fe48009251416c750aca39,1.0,10205.0,Sigourney Weaver,2.0,Avatar
3,4.0,Col. Quaritch,52fe48009251416c750ac9cf,2.0,32747.0,Stephen Lang,3.0,Avatar
4,5.0,Trudy Chacon,52fe48009251416c750ac9d3,1.0,17647.0,Michelle Rodriguez,4.0,Avatar
...,...,...,...,...,...,...,...,...
3,8.0,Himself,58ce018c9251415a7d016e36,2.0,21315.0,Eric Roberts,3.0,My Date with Drew
4,9.0,Himself,58ce01b99251415a7d016e7d,0.0,2171.0,Griffin Dunne,4.0,My Date with Drew
5,10.0,Himself,58ce01d19251415a8b0168be,2.0,2231.0,Samuel L. Jackson,5.0,My Date with Drew
6,11.0,Himself,58ce01dd9251415a39016580,2.0,14407.0,Matt LeBlanc,6.0,My Date with Drew
